In [28]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [29]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [30]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [31]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  147748
Total Vocab:  46


In [32]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  147648


In [33]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [34]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [35]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [36]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
147648/147648 [==============================] - 528s 4ms/step - loss: 2.9756
Epoch 2/20
147648/147648 [==============================] - 547s 4ms/step - loss: 2.7455
Epoch 3/20
147648/147648 [==============================] - 553s 4ms/step - loss: 2.6339
Epoch 4/20
147648/147648 [==============================] - 553s 4ms/step - loss: 2.5593
Epoch 5/20
147648/147648 [==============================] - 569s 4ms/step - loss: 2.4847
Epoch 6/20
147648/147648 [==============================] - 567s 4ms/step - loss: 2.4321
Epoch 7/20
147648/147648 [==============================] - 650s 4ms/step - loss: 2.3741
Epoch 8/20
147648/147648 [==============================] - 671s 5ms/step - loss: 2.3267
Epoch 9/20
147648/147648 [==============================] - 625s 4ms/step - loss: 2.2803
Epoch 10/20
147648/147648 [==============================] - 581s 4ms/step - loss: 2.2348
Epoch 11/20
147648/147648 [==============================] - 572s 4ms/step - loss: 2.2056
Epoch 12/20
147648/

In [37]:
# load the network weights
filename = "weights-improvement-20-1.8994.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [39]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" 
down, down, down. would the fall never come to an end! 'i wonder how
many miles i've fallen by th "
e sooe ' she said to herself, 'ant the soont
then i vanu to the tou th the whut toee ' 

'ie in tou think toe tait th the soitte'' said the caterpillar.

'ie toat so the soitteen,' said the caterpillar.

'i den't seinkngg the sam ' said the cotmouse, and the was a gottle
tame to be a tald tf the tooe. '
'


chapter vii the soeeterg a 
iate aree aalet a
are oo sne to tea to tee so tee to
toeere''

'ie in tou think toe ' said the cotmouse, and the was a gott eiri uiic
seterte to tei the was hnr a mong tine tf the was so the thitg thit
whsh and soine and thene thte the whst oo the care and the corrouse 
tas the had betnr the sabbit woile and thene thse the was all she 
sabe tar an a lote of thitg that she was all she was oo the tan oo
tae in a lorg ofttle thieg what she was all she was oo the tas oo the
crore of thet her an the could oo the tas and the was oo the thneg 
arolous the